In [2]:
import glob
import json
import ipaddress

In [2]:
def read_hosts(path):
    nws = set()
    for h in glob.glob(path):
        with open(h) as fp:
            content = json.load(fp)
            nws.add(ipaddress.ip_network(content['hostInterfaces']['eth0']['prefix'], strict=False))
    return nws

In [3]:
def find_in_lines(needle, search):
    diff_specs = []
    for i, l in enumerate(search.splitlines() if type(search) is str else search):
        # print(needle.__str__())
        if needle.__str__() in l:
            diff_specs.append(l)
            # print(f"Found {needle} in line {i}:", l)
    return diff_specs

In [4]:
from pybatfish.client.session import Session
def find_config_prefixes(path):
  bf = Session("127.0.0.1", 9997, 9996)
  bf.set_network("test")
  bf.init_snapshot(path, name="test", overwrite=True)

  # Get the prefixes from the config
  # interfaces = bf.q.interfaceProperties().answer().frame()
  # print(dir(bf.q))
  dat = set()
  interfaces = bf.q.interfaceProperties().answer().frame()

  for pfxs in interfaces['All_Prefixes']:
    # print(pfxs)
    for pfx in pfxs:
      dat.add(ipaddress.ip_network(pfx, strict=False))

  return dat
  # print(interfaces)
  # print(path)
  

In [12]:
NETWORKS_DIR = "/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/NetworkB-university"
networks = glob.glob(NETWORKS_DIR + "/*")
networks.remove(f"{NETWORKS_DIR}/origin")
networks.insert(0, f"{NETWORKS_DIR}/origin")
print("network_name", "specs#","kept#","false_positive#","false_negative#","#false_negative_fake_dsts","#false_negative_fake_dsts","#hosts", sep='\t')
orig_config_prefixes = set()
orig_host_prefixes = set()

for nw in networks:
    nw_name = nw.split("/")[-1]
    find_config_prefixes(nw)
    # if "strawman" in nw_name:
    #     continue
    # read host prefixes
    if nw_name == "origin":
        orig_host_prefixes = read_hosts(nw + "/hosts/host*.json")
        orig_config_prefixes = find_config_prefixes(nw)
        # print(orig_host_prefixes)
        # print(orig_config_prefixes)
    nw_host_pfxs = read_hosts(nw + "/hosts/host*.json")
    nw_config_prefixes = find_config_prefixes(nw)
    with open(NETWORKS_DIR + f"/{nw_name}/policies.csv", 'r') as fp:
        nw_content = fp.read().__str__()
    with open(NETWORKS_DIR + f"/origin/policies.csv", 'r') as fp:
        origin_content = fp.read().__str__()
    
    origin_policies = set(origin_content.splitlines()[1:])
    nw_policies = set(nw_content.splitlines()[1:])
    fpos = nw_policies - origin_policies
    fneg = origin_policies - nw_policies
    kept = origin_policies & nw_policies

    # fpos and fneg introduced by fake hosts
    fneg_host_count = 0
    fpos_host_count = 0
    for pfx in (nw_host_pfxs - orig_host_prefixes).union(nw_config_prefixes - orig_config_prefixes):
        diff_specs = find_in_lines(pfx, list(fneg))
        fneg_host_count += len(diff_specs)
        diff_specs = find_in_lines(pfx, list(fpos))
        # print(diff_specs)
        fpos_host_count += len(diff_specs)

    # fpos and fneg introduced by fake links
    # fneg_link_count = 0
    # fpos_link_count = 0
    # for pfx in nw_config_prefixes - orig_config_prefixes:
    #     diff_specs = find_in_lines(pfx, list(fneg))
    #     fneg_link_count += len(diff_specs)
    #     diff_specs = find_in_lines(pfx, list(fpos))
    #     # print(diff_specs)
    #     fpos_link_count += len(diff_specs)
    
    # fpos_fake_dst_count = fpos_link_count + fpos_host_count
    # fneg_fake_dst_count = fneg_link_count + fneg_host_count

    with open(NETWORKS_DIR + f"/{nw_name}/policies.csv") as fp:
        specs_count = len(fp.read().splitlines()) - 1
    print(nw_name, specs_count, len(kept), len(fpos), len(fneg), fneg_host_count, fpos_host_count, len(nw_host_pfxs), sep="\t")
    # break

network_name	specs#	kept#	false_positive#	false_negative#	#false_negative_fake_dsts	#false_negative_fake_dsts	#hosts
origin	64337	64337	0	0	0	0	36
anonym-network-k2	79803	47361	32442	16976	0	20560	72
strawman-1-2-2	72297	44055	28242	20282	0	20560	72
anonym-network-k6-4-w-noise	82924	29930	52994	34407	0	42437	144
anonym-network-k2-2-w-noise	51030	26406	24624	37931	0	13111	72
strawman-1	72297	44055	28242	20282	0	20560	72
anonym-network-k6-2-w-noise	52713	30331	22382	34006	0	13273	72
strawman-1-6-2	64295	39548	24747	24789	0	18316	72


In [ ]:
def get_trace_accepted_count(traces):
    accepted_count = 0
    for t in traces[0]:
        if t.disposition == "ACCEPTED":
            accepted_count += 1
    return accepted_count
# print(accepted_count)
# a.Traces[0][0].dict()

In [ ]:
NETWORKS_DIR = "/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/fattree04-ospf"
networks = glob.glob(NETWORKS_DIR + "/*")
networks.remove(f"{NETWORKS_DIR}/origin")
networks.insert(0, f"{NETWORKS_DIR}/origin")
from pybatfish.datamodel.flow import PathConstraints
from pybatfish.client.session import Session, HeaderConstraints
a = None
bf = None
for nw in networks:
    nw_name = nw.split("/")[-1]
    print(nw_name)

    bf = Session("127.0.0.1", 9997, 9996)
    bf.set_network("test")
    bf.init_snapshot(nw, name="test", overwrite=True)

    # verify specs one by one
    with open(nw + "/policies.csv", 'r') as fp:
        nw_specs_content = fp.read().__str__().splitlines()

    specs_count = 0
    holds_count = 0
    holds_not_count = 0
    i = 0
    holds_specs = []
    holds_not_specs = []
    specs_count = len(nw_specs_content) - 1
    
    for spec in nw_specs_content[1:]:
        # print(spec)
        i += 1
        print(f" - {i}/{specs_count}")
        policy_type = spec.split(",")[0]
        src_name = spec.split(",")[3]
        dst_name = spec.split(",")[-2].split(':')[0][1:]
        # if "LoadBalancingSimple" not in policy_type:
        #     continue
        # if src_name != 'e0-1' or dst_name != 'e2-1':
        #     continue
        dst_pfx = spec.split(",")[1]
        # print(policy_type, src_name, dst_name, dst_pfx)
        # result = bf.q.traceroute(startLocation='@enter(as2border1[GigabitEthernet2/0])', headers=HeaderConstraints(dstIps='2.34.201.10', srcIps='8.8.8.8')).answer().frame()
        result = bf.q.reachability(pathConstraints=PathConstraints(startLocation = f'{src_name}'), headers=HeaderConstraints(dstIps=dst_pfx, srcIps='0.0.0.0/0'), actions='SUCCESS').answer().frame()
        # trace = bf.q.traceroute(startLocation="host1", headers=HeaderConstraints(dstIps=spec.split(",")[0])).answer().frame()

        policy_holds = False
        # specs_count += 1

        if policy_type == "PolicyType.Waypoint":
            pass

        if policy_type == "PolicyType.LoadBalancingSimple":
            accepted = get_trace_accepted_count(result.Traces)
            if accepted == int(spec.split(",")[2]):
                policy_holds = True
            pass

        if policy_type == "PolicyType.Reachability":
            accepted = get_trace_accepted_count(result.Traces)
            if accepted > 0:
                policy_holds = True
            pass

        if policy_holds:
            holds_count += 1
            holds_specs.append(spec)
        else:
            holds_not_count += 1
            print("NotHolds", spec)
            holds_not_specs.append(spec)

        with open(f"{nw}/policies-holds.csv", "w+") as fp:
            fp.write("\n".join(holds_specs))
        with open(f"{nw}/policies-holds-not.csv", "w+") as fp:
            fp.write("\n".join(holds_not_specs))
        
        a = result
        # break
    break

origin


 - 1/1920
 - 2/1921
 - 3/1922
 - 4/1923
 - 5/1924
 - 6/1925
 - 7/1926
 - 8/1927
 - 9/1928
 - 10/1929
 - 11/1930
 - 12/1931
 - 13/1932
 - 14/1933
 - 15/1934
 - 16/1935
 - 17/1936
 - 18/1937
 - 19/1938
 - 20/1939
 - 21/1940
 - 22/1941


KeyboardInterrupt: 

In [25]:
bf.q.traceroute(startLocation='@enter(er1[GigabitEthernet2/0])', headers=HeaderConstraints(dstIps='